# Stock screener

This notebook introduces a new idea of screening for stocks based on calculating momentum. Basically,
averages of close prices in sequential windows (in this case I choose windows of 90 days = 1 quarter) are compared against each other. Window 0 is the latest available time window. Window 1 is 90 days before
window 0 and so on. If average close price of window 1 is larger than window 0 then momentum -1, otherwise momentum +1. Final sum of momentum is calculated. If momentum <0 then the stock is bear,  otherwise bull.
My picking criteria in this case is to choose tech stocks with ex-dividend != 0, momentum of 90-day window over 5 years is negative, or momentum of 30-day window over 1 year period is positive, or momentum of 1-day window over 3 month period is positive.

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import datetime as dt

In [2]:
# Download Nasdaq end of day data. This  data source categorizes0 stocks into industries and sectors
all = pd.read_csv("C:\\Dropbox\\nasdaq_screener_1640752808062.csv", header = 0)
all[['Sector','Industry']] = all[['Sector','Industry']].fillna('Unclassified')
all

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$159.18,0.44,0.277%,4.807249e+10,United States,1999.0,764279,Capital Goods,Electrical Products
1,AA,Alcoa Corporation Common Stock,$59.19,-0.64,-1.07%,1.107464e+10,NaN,2016.0,6000817,Basic Industries,Metal Fabrications
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.74,0.00,0.00%,1.217500e+09,NaN,2021.0,8352,Finance,Business Services
3,AACG,ATA Creativity Global American Depositary Shares,$1.17,-0.01,-0.847%,3.671100e+07,China,NaN,26754,Miscellaneous,Service to the Health Industry
4,AACI,Armada Acquisition Corp. I Common Stock,$9.80,0.00,0.00%,2.029531e+08,United States,2021.0,15682,Unclassified,Unclassified
...,...,...,...,...,...,...,...,...,...,...,...
8282,ZWS,Zurn Water Solutions Corporation Common Stock,$36.66,0.19,0.521%,4.448673e+09,United States,2012.0,415326,Public Utilities,Water Supply
8283,ZY,Zymergen Inc. Common Stock,$7.10,-0.06,-0.838%,7.270456e+08,United States,2021.0,636193,Basic Industries,Industrial Specialties
8284,ZYME,Zymeworks Inc. Common Shares,$15.76,-0.06,-0.379%,7.336905e+08,Canada,2017.0,299830,Unclassified,Unclassified
8285,ZYNE,Zynerba Pharmaceuticals Inc. Common Stock,$3.05,-0.11,-3.481%,1.257135e+08,United States,2015.0,935491,Health Care,Biotechnology: Pharmaceutical Preparations


In [3]:
# Check to see which sectors are available
all['Sector'].unique()

array(['Capital Goods', 'Basic Industries', 'Finance', 'Miscellaneous',
       'Unclassified', 'Health Care', 'Consumer Services',
       'Transportation', 'Technology', 'Consumer Durables',
       'Consumer Non-Durables', 'Energy', 'Public Utilities'],
      dtype=object)

In [4]:
# Slice Technology sector
tech = all['Symbol'].loc[all['Sector'] == 'Technology']

In [5]:
# Load wiki data which contain historical time series data of all US stocks
wiki = pd.read_csv("C:\\Users\\daile\\Documents\\WIKI_PRICES_212b326a081eacca455e13140d7bb9db.csv", header = 0)

In [6]:
# Slice technology stocks in wiki data
wikitech = pd.DataFrame()
for i in tech:
    if i in wiki['ticker'].values:
        wikitech = wikitech.append(wiki[wiki['ticker'] == i])
wikitech

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
17894,AAOI,2013-09-26,10.00,10.090,9.37,9.96,946000.0,0.0,1.0,10.00,10.090,9.37,9.96,946000.0
17895,AAOI,2013-09-27,10.44,10.440,10.00,10.10,253300.0,0.0,1.0,10.44,10.440,10.00,10.10,253300.0
17896,AAOI,2013-09-30,10.00,10.180,9.71,10.00,84900.0,0.0,1.0,10.00,10.180,9.71,10.00,84900.0
17897,AAOI,2013-10-01,9.95,10.024,9.92,10.00,74500.0,0.0,1.0,9.95,10.024,9.92,10.00,74500.0
17898,AAOI,2013-10-02,9.99,10.000,9.89,9.97,94000.0,0.0,1.0,9.99,10.000,9.89,9.97,94000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15377357,ZNGA,2018-03-21,3.85,3.940,3.81,3.94,17590899.0,0.0,1.0,3.85,3.940,3.81,3.94,17590899.0
15377358,ZNGA,2018-03-22,3.88,3.940,3.80,3.81,14379224.0,0.0,1.0,3.88,3.940,3.80,3.81,14379224.0
15377359,ZNGA,2018-03-23,3.81,3.860,3.79,3.80,9850955.0,0.0,1.0,3.81,3.860,3.79,3.80,9850955.0
15377360,ZNGA,2018-03-26,3.85,3.860,3.78,3.85,9075940.0,0.0,1.0,3.85,3.860,3.78,3.85,9075940.0


In [7]:
# Function to get data of a particular ticker provided by wiki
def getdf(wikidf,ticker):
    df = wikidf[wikidf['ticker']== ticker]
    df = df.set_index('date')
    df.index = df.index.astype('datetime64[ns]')
    print(f'Data for {ticker}')
    return df

In [8]:
''' Function to calculate rolling mean. This function takes time window and period as arguments and
return dataframe with calculated means and momentum '''
def rollingmean(sdf,window, period):
    sdf = sdf[['open','close','volume','ex-dividend','split_ratio','adj_open','adj_close','adj_volume']]
    sdfmean = []
    date = []
    momentum = []
    for i in range(2,period):
        max0 = sdf[pd.to_datetime(sdf.index)>(pd.to_datetime(sdf.index.max()+dt.timedelta(days = -window)))]
        max1 = sdf[pd.to_datetime(sdf.index)>(pd.to_datetime(sdf.index.max()+dt.timedelta(days = -window*(i-1))))].index.max()
        max2 = sdf[pd.to_datetime(sdf.index)>(pd.to_datetime(max1+dt.timedelta(days = -window*i)))]
        sdfmean.append(max0.mean())
        sdfmean.append(max2.mean())
        if (max2['close'].mean() < max0['close'].mean()):
            momentum.append(1)
        else:
            momentum.append(-1)
        date.append(max1)
        date.append(max2.index[0])
    df = pd.DataFrame(sdfmean)
    df['date'] = date
    df['date'] = df['date'].astype('datetime64[ns]')
    df = df.set_index('date')
    df = df.sort_index()
    df = df.drop_duplicates()
    df = df.round(4)
    return df, momentum

In [9]:
'''Screening for tech stocks that have ex-dividend != 0, momentum of 90-day window over 5 years
 is negative, while momentum of 30-day window over 1 year period is positive, momentum of 1-day window
 over 3 month period is positive'''
techlist = []
for i in tech:
    df = getdf(wikitech,i)
    if df['volume'].any()!=0:
        mean, momentum90 = rollingmean(df, 90, 20)
        print(sum(momentum90))
        if sum(momentum90) <0 and sum(mean['ex-dividend']) > 0:
            techlist.append(i)
    else:
        pass

Data for AAOI
-2
Data for AAPL
14
Data for ABM
-2
Data for ACEV
Data for ACEVU
Data for ACEVW
Data for ACIW
12
Data for ACLS
14
Data for ACMR
Data for ACN
18
Data for ACVA
Data for ACY
Data for ADBE
18
Data for ADI
18
Data for ADSK
18
Data for ADTH
Data for ADV
Data for ADVWW
Data for AER
Data for AEYE
Data for AFRM
Data for AGIL
Data for AGILW
Data for AGMH
Data for AGYS
16
Data for AI
-18
Data for AIP
Data for AIRG
Data for AL
18
Data for ALF
Data for ALFIW
Data for ALGM
Data for ALIT
Data for ALKT
Data for ALOT
Data for ALRM
Data for ALTG
Data for ALTR
-18
Data for AMBA
-16
Data for AMD
12
Data for AMKR
10
Data for AMPL
Data for AMST
Data for AMSWA
18
Data for AMWL
Data for ANET
Data for ANGI
18
Data for ANSS
18
Data for ANTE
Data for AOSL
2
Data for API
Data for APP
-18
Data for APPF
Data for APPN
Data for APPS
Data for ARNC
18
Data for ARRY
18
Data for ARW
18
Data for ASAN
Data for ASGN
18
Data for ASML
Data for ASUR
Data for ASX
Data for ASYS
Data for ATEN
-18
Data for ATHM
Data 

In [10]:
#List of tickers that satisfy conditions
techlist

['ABM',
 'AI',
 'ALTR',
 'CCO',
 'CPSI',
 'DBD',
 'DMRC',
 'EVTC',
 'HCP',
 'NCMI',
 'NSP',
 'OMC',
 'RCII',
 'SE',
 'SMRT',
 'SPNS']

In [11]:
# Check for dividend and plot the whole data for verification
for i in techlist:
    df = getdf(wikitech,i)
    for j in df['ex-dividend']:
        if j !=0:
            print(f'expected dividend of {i} is {j}')
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index,y=df['close'], name = 'close'))
    fig.add_trace(go.Scatter(x=df.index,y = df['adj_close'], name = 'adjusted close'))
    fig.update_layout(title = i, template = 'plotly_dark')
    fig.show()

"for i in techlist:\n    df = getdf(wikitech,i)\n    for j in df['ex-dividend']:\n        if j !=0:\n            print(f'expected dividend of {i} is {j}')\n    fig = go.Figure()\n    fig.add_trace(go.Scatter(x=df.index,y=df['close'], name = 'close'))\n    fig.add_trace(go.Scatter(x=df.index,y = df['adj_close'], name = 'adjusted close'))\n    fig.update_layout(title = i, template = 'plotly_dark')\n    fig.show()"

In [12]:
techlist2 = []
for i in tech:
    df = getdf(wikitech,i)
    if df['volume'].any()!=0:
        mean, momentum30 = rollingmean(df, 30, 12)
        print(sum(momentum30))
        if sum(momentum30) > 0 and sum(mean['ex-dividend']) > 0:
            techlist2.append(i)
    else:
        pass

Data for AAOI
-10
Data for AAPL
10
Data for ABM
-10
Data for ACEV
Data for ACEVU
Data for ACEVW
Data for ACIW
10
Data for ACLS
-6
Data for ACMR
Data for ACN
10
Data for ACVA
Data for ACY
Data for ADBE
10
Data for ADI
10
Data for ADSK
10
Data for ADTH
Data for ADV
Data for ADVWW
Data for AER
Data for AEYE
Data for AFRM
Data for AGIL
Data for AGILW
Data for AGMH
Data for AGYS
-4
Data for AI
-6
Data for AIP
Data for AIRG
Data for AL
-4
Data for ALF
Data for ALFIW
Data for ALGM
Data for ALIT
Data for ALKT
Data for ALOT
Data for ALRM
Data for ALTG
Data for ALTR
-10
Data for AMBA
-8
Data for AMD
-6
Data for AMKR
10
Data for AMPL
Data for AMST
Data for AMSWA
10
Data for AMWL
Data for ANET
Data for ANGI
10
Data for ANSS
10
Data for ANTE
Data for AOSL
-8
Data for API
Data for APP
-10
Data for APPF
Data for APPN
Data for APPS
Data for ARNC
-10
Data for ARRY
10
Data for ARW
-6
Data for ASAN
Data for ASGN
10
Data for ASML
Data for ASUR
Data for ASX
Data for ASYS
Data for ATEN
-8
Data for ATHM
Data

In [13]:
techlist2

['AAPL',
 'ACN',
 'ADI',
 'AMSWA',
 'AVT',
 'BBSI',
 'BLKB',
 'CCMP',
 'CMTL',
 'CSCO',
 'CSGS',
 'CTSH',
 'DLB',
 'DLX',
 'DOX',
 'DV',
 'DXC',
 'EBIX',
 'EFX',
 'ENTG',
 'EVER',
 'EVTC',
 'FSS',
 'HCKT',
 'HEES',
 'HPQ',
 'HSII',
 'IBM',
 'INTC',
 'INTU',
 'IPG',
 'JBL',
 'JKHY',
 'KAI',
 'KELYA',
 'KFRC',
 'KFY',
 'LDOS',
 'LFUS',
 'LRCX',
 'MANT',
 'MCHP',
 'MCO',
 'MGRC',
 'MORN',
 'MPWR',
 'MRVL',
 'MSCI',
 'MSFT',
 'MSI',
 'NATI',
 'NTAP',
 'NVDA',
 'ORCL',
 'PEGA',
 'RELL',
 'RHI',
 'ROL',
 'SAIC',
 'SPGI',
 'SSNC',
 'STX',
 'SWKS',
 'TEL',
 'TXN',
 'WDC',
 'XLNX']

In [14]:
# Check for dividend and plot the whole data for verification
for i in techlist2:
    df = getdf(wikitech,i)
    for j in df['ex-dividend']:
        if j !=0:
            print(f'expected dividend of {i} is {j}')
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index,y=df['close'], name = 'close'))
    fig.add_trace(go.Scatter(x=df.index,y = df['adj_close'], name = 'adjusted close'))
    fig.update_layout(title = i, template = 'plotly_dark')
    fig.show()

"for i in techlist2:\n    df = getdf(wikitech,i)\n    for j in df['ex-dividend']:\n        if j !=0:\n            print(f'expected dividend of {i} is {j}')\n    fig = go.Figure()\n    fig.add_trace(go.Scatter(x=df.index,y=df['close'], name = 'close'))\n    fig.add_trace(go.Scatter(x=df.index,y = df['adj_close'], name = 'adjusted close'))\n    fig.update_layout(title = i, template = 'plotly_dark')\n    fig.show()"

In [15]:
techlist3 = []
for i in tech:
    df = getdf(wikitech,i)
    if df['volume'].any()!=0:
        mean, momentum1 = rollingmean(df, 1, 90)
        print(sum(momentum1))
        if sum(momentum1) > 0 and sum(mean['ex-dividend']) > 0:
            techlist3.append(i)
    else:
        pass

Data for AAOI
-82
Data for AAPL
-88
Data for ABM
-88
Data for ACEV
Data for ACEVU
Data for ACEVW
Data for ACIW
-48
Data for ACLS
-88
Data for ACMR
Data for ACN
-88
Data for ACVA
Data for ACY
Data for ADBE
6
Data for ADI
-14
Data for ADSK
24
Data for ADTH
Data for ADV
Data for ADVWW
Data for AER
Data for AEYE
Data for AFRM
Data for AGIL
Data for AGILW
Data for AGMH
Data for AGYS
4
Data for AI
88
Data for AIP
Data for AIRG
Data for AL
-88
Data for ALF
Data for ALFIW
Data for ALGM
Data for ALIT
Data for ALKT
Data for ALOT
Data for ALRM
Data for ALTG
Data for ALTR
88
Data for AMBA
-88
Data for AMD
-88
Data for AMKR
-88
Data for AMPL
Data for AMST
Data for AMSWA
-88
Data for AMWL
Data for ANET
Data for ANGI
-88
Data for ANSS
-88
Data for ANTE
Data for AOSL
-88
Data for API
Data for APP
-88
Data for APPF
Data for APPN
Data for APPS
Data for ARNC
-86
Data for ARRY
-88
Data for ARW
-86
Data for ASAN
Data for ASGN
6
Data for ASML
Data for ASUR
Data for ASX
Data for ASYS
Data for ATEN
-88
Data f

In [16]:
techlist3

['CYN', 'EVER']

In [17]:
# Check for dividend and plot the whole data for verification
for i in techlist3:
    df = getdf(wikitech,i)
    for j in df['ex-dividend']:
        if j !=0:
            print(f'expected dividend of {i} is {j}')
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index,y=df['close'], name = 'close'))
    fig.add_trace(go.Scatter(x=df.index,y = df['adj_close'], name = 'adjusted close'))
    fig.update_layout(title = i, template = 'plotly_dark')
    fig.show()

"for i in techlist3:\n    df = getdf(wikitech,i)\n    for j in df['ex-dividend']:\n        if j !=0:\n            print(f'expected dividend of {i} is {j}')\n    fig = go.Figure()\n    fig.add_trace(go.Scatter(x=df.index,y=df['close'], name = 'close'))\n    fig.add_trace(go.Scatter(x=df.index,y = df['adj_close'], name = 'adjusted close'))\n    fig.update_layout(title = i, template = 'plotly_dark')\n    fig.show()"